In [ ]:
import os
import pandas as pd
import numpy as np
import json
import seaborn as sns
import funcTest as ft
import funcData as fd

In [ ]:
path_rawdata = 'F://TFG//datasets/raw_datasets//'
path_train = 'F://TFG//datasets//data_train//'
path = 'F:/TFG/datasets/nature-dataset/'


rawdata = pd.read_csv(path_rawdata+'rawDB.csv')
footdata = pd.read_csv(path_rawdata+'fdataDB.csv')
partidos_raw = pd.read_json(path_rawdata+'RAW_partidos.json')

### DATOS EN BRUTO DE NATURE

In [ ]:
rawdata[:5]

### EXPLORACION

In [ ]:
match = rawdata.loc[0]
match

In [ ]:
formacion = match['formation_home']

In [ ]:
path = 'F:/TFG/datasets/nature-dataset/'

matches={}
nations = ['Italy','England','Germany','France','Spain','European_Championship','World_Cup']
for nation in nations:
    with open(path + '/matches/matches_%s.json' %nation) as json_data:
        matches[nation] = json.load(json_data)
        
# loading the events data
events={}
nations = ['Italy','England','Germany','France','Spain','European_Championship','World_Cup']
for nation in nations:
    with open(path + '/events/events_%s.json' %nation) as json_data:
        events[nation] = json.load(json_data)

In [ ]:
# loading the players data
path = 'F:/TFG/datasets/nature-dataset/'

players={}
with open(path + '/players.json',encoding='utf-8') as json_data:
    players = json.load(json_data)
    
# loading the teams data
teams={}
with open(path + '/teams.json',encoding='utf-8') as json_data:
    teams = json.load(json_data)

In [ ]:
partidos = []
eventos  = []

paises = ['Italy','England','Germany','France','Spain']

for pais in paises:
    for m in matches[pais]:
        partidos.append(m)
    for e in events[pais]:
        eventos.append(e)
        
partidos = np.array(partidos,dtype=dict)
eventos = np.array(eventos,dtype=dict)
partidos.shape

In [ ]:
tags = pd.read_csv(path+'tags2name.csv')
tags.set_index('Tag',inplace=True,drop=True)

In [ ]:
# eventosDF = pd.DataFrame(list(eventos),columns=list(eventos[0].keys()))
eventosDF = pd.read_json(path_rawdata+'eventos_raw.json')
eventosDF.set_index('id',inplace=True)
teams = pd.read_csv(path_rawdata+'teams.csv',index_col='wyId',delimiter=';')
players = pd.read_csv(path_rawdata+'players.csv',index_col='wyId',delimiter=';')

In [ ]:
eventos_dict = list(set(zip(eventosDF.eventId,eventosDF.eventName)))
eventos_dict = {v:k for k,v in eventos_dict}
eventos_dict

In [ ]:
passes = eventosDF[eventosDF.eventId == 8]
shots = eventosDF[eventosDF.eventId == eventos_dict['Shot']]

In [ ]:
shots

#### VAMOS A PROBAR CON ESTADISTICAS DE UN EQUIPO

En este caso usaremos el Atalanta como modo de ejemplo

In [ ]:
teams.name[3172]

In [ ]:
shots_atalanta = shots[shots.teamId == 3172] # atalanta = 3172
passes_atalanta = passes[passes.teamId == 3172]

### Vamos a ver por ejemplo los jugadores con más disparos:

In [ ]:
shots_atalanta['counter'] = np.ones(len(shots_atalanta))
shots_atalanta.columns

In [ ]:
top5_shooters_atalanta = shots_atalanta.groupby('playerId').sum('counter').sort_values('counter',ascending=0)[:5]
top5_shooters_atalanta

In [ ]:
for p in top5_shooters_atalanta.itertuples():
    pl = players.loc[p.Index]
    print(pl.firstName + ' ' + pl.lastName + ' -> ' + str(p.counter))

#### Jugadores con más porcentaje de acierto de disparo (en Atalanta)

In [ ]:
list(shots_atalanta[:20].tags)

In [ ]:
acc = [int('1801' in tag) for tag in list(shots_atalanta.tags)]

In [ ]:
shots_atalanta['accuracy'] = acc

In [ ]:
top5_acc_atalanta = shots_atalanta.groupby('playerId').mean('accuracy').sort_values('accuracy',ascending=0)[:5]
top5_acc_atalanta

In [ ]:
for p in top5_acc_atalanta.itertuples():
    pl = players.loc[p.Index]
    print(pl.firstName + ' ' + pl.lastName + ' -> ' + str(p.accuracy))

#### Acierto disparos total del equipo (Atalanta)

In [ ]:
print(f'Acierto total del equipo: {np.array(acc).mean()}')

# DATASET EVENTOS

In [ ]:
# matches = pd.read_csv(path_rawdata+'Nature_wStatsDB_v2.csv',index_col='dateutc')
# matches.index = pd.to_datetime(matches.index)
# matches[:4]

matches = pd.read_csv(path_rawdata+'Nature_wStatsDB_v2.csv')
matches.set_index('wyId',inplace=True)
matches[:4]

In [ ]:
matches.dateutc = pd.to_datetime(matches.dateutc)

Vamos a limpiar el dataset de disparos:
- Posiciones: en un array [y0,x0,y1,x1]
- Accuracy: 1 o 0
- Goal: 1 o 0

In [ ]:
# posiciones y tags en formato array
shots['positions'] = [[p.positions[0]['y'],p.positions[0]['x'],p.positions[1]['y'],p.positions[1]['y'],] for p in shots.itertuples()]
shots['tags'] = np.array([[t['id'] for t in s.tags] for s in shots.itertuples()])

In [ ]:
posDF = pd.DataFrame(shots["positions"].to_list(), columns=['y0', 'x0','y1','x1'])
posDF.shape

In [ ]:
shots.reset_index(inplace=True)

In [ ]:
posDF

In [ ]:
for c in list(posDF.columns):
    shots[c] = posDF[c]

In [ ]:
shots = shots.drop('positions',axis=1)

In [ ]:
shots['accuracy'] = [int(1801 in t) for t in shots.tags]
shots['goal'] = [int(101 in t) for t in shots.tags]

In [ ]:
shots

In [ ]:
# shots.to_json(path_rawdata+'shots.json')
shots = pd.read_json(path_rawdata+'shots.json')

#### Funcion que selecciona los partidos del ultimo mes

In [ ]:
def n_matches(db,n):
    ...

### 5 jugadores más disparan cada equipo

De cada partido calculamos los 5 jugadores que mas disparan. Hacemos un dataframe con una columna donde salgan esos 5 jugadores.
Más tarde, para cada equipo podemos sacar los 5 jugadores que más aparezcan en cada lista, en cada partido.

Convendria asignar pesos, ya que alomejor un jugador abarca el 40% de los disparos de un equipo, y por lo tanto no tiene la misma importancia que el resto de los 5 jugadores que más disparan.
A lo mejor podriamos para ello sacar la distribucion de disparos de los jugadores, para calcular el peso de cada jugador.

P.e. La calidad (en cuanto a disparos) de un jugador puede depender de cuantos disparos x partido

Podriamos por ejemplo representar en una gráfica donde x= # jugadores importantes (mas disparan) que le faltan al equipo, y donde y = # goles marcados o resultado (v-e-d) p.e.

In [ ]:
shots[-3:]

### Estadisticas por equipo de disparos por partido.

Resumen de toda la temporada:

In [ ]:
team_stats = shots.groupby('teamId').agg(
    y0=('y0','mean'),x0=('x0','mean'),y1=('y1','mean'),x1=('x1','mean'),
    shots_count=('accuracy','count'),acc_count=('accuracy','sum'),
    acc_avg=('accuracy','mean'),
    goal_count=('goal','sum'),goal_avg=('goal','mean'),
)
team_stats.sort_values('goal_count',ascending=False)

In [ ]:
# hay algunos casos concretos que solo un equipo realiza disparos durante un partido
np.unique(shots.set_index('matchId').loc[2576281].teamId)

In [ ]:
shots_match = shots.groupby(['matchId','teamId']).agg(
    y0=('y0','mean'),x0=('x0','mean'),
    acc_avg=('accuracy','mean'),
    goal_avg=('goal','mean'),
)

shots_match

In [ ]:
sm_aux1 = shots_match.reset_index()[::2]
sm_aux2 = shots_match.reset_index()[1::2]

In [ ]:
shots_match = pd.merge(sm_aux1,sm_aux2,on='matchId',how='outer',suffixes=('_sh_H','_sh_A')).set_index('matchId')
shots_match


In [ ]:
shots_match[shots_match.isna().any(axis=1)].reset_index()

In [ ]:
matches['matchId'] = matches.index.to_list()
# matches.drop('wyId',axis=1,inplace=True)

In [ ]:
for row in shots_match[shots_match.isna().any(axis=1)].reset_index().itertuples():
    shots_match.loc[row.matchId].teamId_sh1 = matches.loc[row.matchId].teamId_home
    shots_match.loc[row.matchId].teamId_sh2 = matches.loc[row.matchId].teamId_away

In [ ]:
shots_match[shots_match.isna().any(axis=1)].reset_index()

En vez de eliminar las filas con valores NaN vamos a sustituir los valores con el ID del equipo, pero los valores de y0 x0 y de accuracy lo dejamos a NaN
Cuando se utilice en un modelo como input habrá que filtrar dichos valores! Sin embargo, para crear los datos de los partidos posteriores si que es útil tenerlo.

In [ ]:
matches_stats = pd.merge(matches.reset_index(),shots_match.reset_index(),on='matchId',how='inner')
matches_stats.set_index('matchId',inplace=True, drop=True)
matches_stats.drop(['teamId_sh_H', 'teamId_sh_A'],axis=1,inplace=True)

In [ ]:
matches_stats[matches_stats.isna().any(axis=1)]

In [ ]:
# matches_stats.to_csv(path_rawdata+'match_stats.csv',sep=';')
matches_stats = pd.read_csv(path_rawdata+'match_stats.csv',sep=';',index_col='matchId')
# matches_stats.drop(['teamId_sh1', 'teamId_sh2'],axis=1,inplace=True)
matches_stats[:3]

In [ ]:
matches_stats.columns

#### Sacamos los jugadores con mas disparos en un partido

In [ ]:
n = 4
mtc = 2576143
tid = matches.loc[mtc].teamId_home
mask_time = (matches.loc[mtc].dateutc > matches.dateutc)
mask_team_h = (matches.teamId_home == tid)
mask = mask_time & mask_team_h
# mask_team_a = (matches.teamId_away == tid)

# TODO: hacer loop que recorra cada partido, y hacer el proceso para los dos equipos de cada partido

In [ ]:
partidos_anteriores = matches[mask][:n].index.to_list()

In [ ]:
players_shots = shots[shots.matchId == partidos_anteriores[0]].playerId.to_numpy()
np.argsort(np.bincount(players_shots))[::-1][:4]

In [ ]:
def ult_partidos(npj,nj=4):

    partidos_h = {}
    partidos_a = {}

    for mtc in matches.index.to_list():
        tid_h, tid_a = matches.loc[mtc].teamId_home, matches.loc[mtc].teamId_away
        mask_time = (matches.loc[mtc].dateutc > matches.dateutc)
        mask_team_h, mask_team_a = (matches.teamId_home == tid_h), (matches.teamId_away == tid_a)
        mask_h, mask_a = mask_time & mask_team_h, mask_time & mask_team_a
        # hecho para que solo coja los ultimos partidos en casa si el partido actual es en casa. Igual para visitante.


        if sum(mask_h) >= npj:
            partidos_anteriores_h = (matches[mask_h][:npj].index).to_numpy()
            partidos_anteriores_a = (matches[mask_a][:npj].index).to_numpy()
            # # 
            # # calcular top shooters:
            # matches_stats.loc[mtc].top_shooters_h = np.argsort(np.bincount(shots[shots.matchId.isin(partidos_anteriores_h)].playerId))[::-1][:nj]
            # matches_stats.loc[mtc].top_shooters_a = np.argsort(np.bincount(shots[shots.matchId.isin(partidos_anteriores_a)].playerId))[::-1][:nj]
        else:
            partidos_anteriores_h, partidos_anteriores_a = np.nan, np.nan

        partidos_h[mtc], partidos_a[mtc] = partidos_anteriores_h, partidos_anteriores_a

        # jugadores mas disparos

    return partidos_h, partidos_a

In [ ]:
matches[matches.teamId_home == 3162][:10]

In [ ]:
partidos_h, partidos_a = ult_partidos(5,4)

In [ ]:
np.argsort(np.bincount(shots[shots.matchId.isin(partidos_h[2576335])].playerId))[::-1][:4]

In [ ]:
matches_stats.shape

In [ ]:
matches.shape

### Repetimos lo mismo pero con los pases

In [ ]:
passes[:3]

In [ ]:
# posiciones y tags en formato array
passes['positions'] = [[p.positions[0]['y'],p.positions[0]['x'],p.positions[1]['y'],p.positions[1]['y'],] for p in passes.itertuples()]
passes['tags'] = [[t['id'] for t in s.tags] for s in passes.itertuples()]

In [ ]:
posDF = pd.DataFrame(passes["positions"].to_list(), columns=['y0', 'x0','y1','x1'])
print(posDF.shape)
passes.reset_index(inplace=True)
posDF

In [ ]:
for c in list(posDF.columns):
    passes[c] = posDF[c]

In [ ]:
subEventDictPass = { id:name for id,name in zip(np.unique(passes.subEventId),np.unique(passes.subEventName)) }
subEventDictPass

In [ ]:
# passes = passes.drop('positions',axis=1)
passes[passes.subEventId == 86]

In [ ]:
passes['accuracy'] = np.array([(1801 in t) for t in passes.tags])*1
passes['keyPass'] = np.array([(302 in t) for t in passes.tags])*1

In [ ]:
passes

In [ ]:
# passes.to_json(path_rawdata+'passes.json')
passes = pd.read_json(path_rawdata+'passes.json')

In [ ]:
passes_match = passes.groupby(['matchId','teamId']).agg(
    y0=('y0','mean'),x0=('x0','mean'),
    y1=('y1','mean'),x1=('x1','mean'),
    acc_avg=('accuracy','mean'),
    keypass=('keyPass','sum'),
)

passes_match

In [ ]:
sm_aux1 = passes_match.reset_index()[::2]
sm_aux2 = passes_match.reset_index()[1::2]

In [ ]:
passes_match = pd.merge(sm_aux1,sm_aux2,on='matchId',how='outer',suffixes=('_ps_H','_ps_A')).set_index('matchId')
passes_match

In [ ]:
passes_match[passes_match.isna().any(axis=1)].reset_index()

# no hay ningun valor indefinido NaN

In [ ]:
matches_stats = pd.merge(matches_stats,passes_match.reset_index(),on='matchId',how='inner')
# matches_stats.set_index('matchId',inplace=True, drop=True)
matches_stats.drop(['teamId_ps_H', 'teamId_ps_A'],axis=1,inplace=True)

In [ ]:
matches_stats.columns

## Total minutos acumulados de la alineacion

Sumamos los minutos jugados en los ultimos N partidos por los jugadores que componen la alineacion inicial del partido

### Crear tabla de las Formaciones

In [ ]:
partidos_raw[:3].teamsData

Vamos a hacer un DataFrame que indique los IDs de los jugadores que juegan y otro DataFrame con los minutos.
Tendremos dos DF que cada posicion concuerda con el jugador y tiempo de juego

In [ ]:
# partidos_raw.set_index('wyId',inplace=True)

In [ ]:
print(partidos_raw.columns)
assert matches.reset_index().wyId.all() == partidos_raw.reset_index().wyId.all()

Creacion de las dos tablas

In [ ]:
x = {'a': 1, 'b': 2}
y = {'d': 10, 'c': 11}

z = {**x, **y}
list(z.values())

In [ ]:
playerIds = np.array(players.index)

In [ ]:
players_match_mins = {pId:{} for pId in playerIds}    # creamos un dict vacio para cada jugador

In [ ]:
matches

In [ ]:
indice          = [] # idmatch_idteam
players_id_mins = []
players_mins    = []

# tambien estamos interesados en mantener un diccionario con todos los minutos que han jugado cada jugador y en que partidos
players_match_mins = {pId:{} for pId in playerIds}    # creamos un dict vacio para cada jugador

for m in matches.reset_index().wyId:
    form_info = partidos_raw.teamsData.loc[m]                                   # seleccionamos el campo de las formaciones
    for t in list(form_info.keys()):                                            # iteramos sobre los dos equipos
        indice.append((m,int(t)))                                               # guardamos en una lista de tuplas lo que seran las keys de la tabla (partido,equipo)
        form_t = form_info[t]['formation']                                      # seleccionamos la formacion de uno de los equipos
        dict_team1 = {pl['playerId']:0 for pl in form_t['bench']}              
        dict_team2 = {pl['playerId']:90 for pl in form_t['lineup']}
        dict_match = {**dict_team1, **dict_team2}
        # añadimos los minutos en el dict de los minutos de cada jugador a lo largo de la temporada
        for pl in form_t['bench']:
            if players_match_mins.get(pl['playerId'],-1) == -1: players_match_mins[pl['playerId']] = {}
            players_match_mins[pl['playerId']][m] = 0
        for pl in form_t['lineup']:
            if players_match_mins.get(pl['playerId'],-1) == -1: players_match_mins[pl['playerId']] = {}
            players_match_mins[pl['playerId']][m] = 90
        # print((m,int(t)))
        if form_t['substitutions'] != 'null':
            for subs in form_t['substitutions']:
                # print(subs['minute'])
                if players_match_mins.get(subs['playerIn'],-1) != -1 and players_match_mins.get(subs['playerOut'],-1) != -1:
                    mins = subs['minute'] if (subs['minute'] < 90) else 90
                    dict_match[subs['playerIn']] = 90-mins
                    players_match_mins[subs['playerIn']][m] = 90-mins  
                    dict_match[subs['playerOut']] = dict_match[subs['playerOut']] - (90-mins)  
                    players_match_mins[subs['playerOut']][m] = dict_match[subs['playerOut']] - (90-mins) 
                

        players_id_mins.append(np.array(list(dict_match.keys())))   
        players_mins.append(np.array(list(dict_match.values())))   

players_id_mins = np.array(players_id_mins)
players_mins = np.array(players_mins)


In [ ]:
players_match_mins[372752][2565608]

In [ ]:
# comprobamos que todos las 3 listas tengan el mismo tamaño
assert len(players_id_mins) == len(players_mins) == len(indice)

In [ ]:
# todos los equipos deberian sumar 990 minutos en un partido entre sus jugadores

for mId,mins in zip(indice,players_mins):
    if np.sum(mins) != 990:
        print(mId, mins)
    # assert np.sum(m) == 11*90

# PARTIDOS QUE NO CUMPLEN LOS MINUTOS:
# (2575965, 3204) [ 0  0  0  0  0  0  0  0  0  0  0  0 56 62 88 90 90 90 90 90 90 90 90  2] -> no estan registrados los IDs de los jugadores que entran al partido
# (2575959, 3158) [ 0  0  0  0  0  0  0  0  0  0  0 74 81 84 90 90 90 90 90 90 90 90  6] -> no estan registrados los IDs de los jugadores que entran al partido

In [ ]:
# partidos_raw.loc[2575959].teamsData

In [ ]:
indice_list = np.array(indice)
indice_list

In [ ]:
# vamos a crear un dataframe con dichos arrays

players_mins_DF = pd.DataFrame({'matchesId': indice_list[:,0], 'teamsId': indice_list[:,1], 'players_id': players_id_mins, 'players_min': players_mins})
players_mins_DF.set_index(['matchesId','teamsId'],drop=True,inplace=True)
players_mins_DF

In [ ]:
players_mins_DF.loc[(2576281, 3173)]

In [ ]:
partidos_raw.loc[2576281]

In [ ]:
print(players_match_mins[393228].values())
print(len(players_match_mins[393228].keys()))


## Vector de caracteristicas para cada equipo, solo teniendo en cuenta los jugadores de la alineacion

- Features defensivas -> tackles, fouls, interception, clearances...
- Features portero -> saves attempts
- Features ataque -> smart passes, cross passes, duels
- Features generales -> missed balls, time loss,  

Vamos a crear estadisticas de cada equipo en cada partido, partiendo de los jugadores del lineup.

Evaluaremos:
- Estadisticas como valor absoluto -> i.e. # smart passes
- Estadisticas por minutos jugados (sin weighted) -> i.e. # smart passes / # minutos jugados
- Estadisticas por minutos jugados (weighted) -> i.e. # smart passes * # minutos jugados / # minutos jugados

In [ ]:
def _2means(m,x):
    print((1/m @ x) / np.sum(1/m), np.mean(x))
    print()


#### VECTOR MINUTOS X EQUIPO

Vamos a crear dos vectores: uno con el numero de minutos acumulados en los últimos N partidos por los jugadores titulares del partido, el otro vector será lo mismo pero dividiendo por el numero total de minutos disputados en los ultimos N partidos (N * 90 mins).

También puede ser interesante crear un vector con unos pesos que multipliquen los minutos disputados. Los pesos representarian los resultados de los últimos N partidos jugados por el jugador.

In [ ]:
# vamos a añadir a matches_stats dos vectores (uno para cada equipo) con el % de minutos disputados de cada equipo

matches_stats.head(3)

In [ ]:
# vamos a calcular los vectores a partir de los últimos npj partidos
npj = 4
partidos_h, partidos_a = ult_partidos(npj=npj)

In [ ]:
players_mins_DF.loc[2576335,3162].players_id[:-11]

In [ ]:
partidos_h[2565632]

In [ ]:
# seleccionamos solo los partidos que tienen 4 partidos anteriores
idxs = np.array(pd.DataFrame(partidos_h).T.reset_index().dropna(axis=0)['index'].to_list())
matches_wUltPartidos = matches_stats.set_index('wyId').loc[idxs]

In [ ]:
# iteramos sobre cada partido de matches_stats y para cada equipo seleccionamos los jugadores titulares.

minsH, minsA = np.zeros(len(matches_wUltPartidos)), np.zeros(len(matches_wUltPartidos))

for i,m in enumerate(matches_wUltPartidos.reset_index().itertuples()):
    # if m.wyId == 2565632: print(m.wyId)
    # los jugadores titulares son los últimos 11 elementos de los arrays 'players_id'
    lineup_h = players_mins_DF.loc[m.wyId, m.teamId_home].players_id[-11:]
    lineup_a = players_mins_DF.loc[m.wyId, m.teamId_away].players_id[-11:]

    for player in lineup_h:
        for part in partidos_h[m.wyId]:
            if part == 2565632: print(player)
            minsH[i] += players_match_mins[player].get(part,0)

    for player in lineup_a:
        for part in partidos_a[m.wyId]:
            minsA[i] += players_match_mins[player].get(part,0)

    # minsH = minsH / (npj*90)
    # minsA = minsA / (npj*90)

    # OPCION CODIGO
    # crear un diccionario grande de los ultimos N partidos (playerId : player_min) y hacer ReduceByKey(add), con lo cual tendremos la suma de minutos de cada jugador
    # a continuación escogeriamos solamente las Keys de los jugadores del lineup y las sumariamos
        

In [ ]:
assert len(minsA) == len(minsH) == len(matches_wUltPartidos)

In [ ]:
matches_wUltPartidos['mins4_H'] = minsH / (npj*11)
matches_wUltPartidos['mins4_A'] = minsA / (npj*11)

In [ ]:
# matches_stats.to_csv(path_rawdata+'match_stats.csv',sep=';')
matches_stats = pd.read_csv(path_rawdata+'match_stats.csv',sep=';',index_col='matchId')
matches_stats[:3]

In [ ]:
matches_stats.mins4_H[2565632]

In [ ]:
matches_wUltPartidos.mins4_H[2565632]

In [ ]:
min(matches_wUltPartidos.mins4_A)

### AÑADIR LOS VECTORES DE LAS DEMÁS CARACTERÍSTICAS

1) Sacamos de cada equipo cuantos disparos/pases/faltas ha hecho en los ultimos N partidos los jugadores del 11 titular, y luego lo dividimos entre el numero de minutos que hayan jugado los titulares.

In [ ]:
passes

In [ ]:
def filterDB(db,matches,players):
    mask_players = False
    for p in players:
        mask_players = mask_players | np.array(db.playerId == p)

    mask_matches = False
    for m in matches:
        mask_matches = mask_matches | np.array(db.matchId == m)

    mask = mask_matches & mask_players
    
    return db[mask]

In [ ]:
matches_stats.head(2)

In [ ]:
partidos_h, partidos_a = ult_partidos(4)

In [ ]:
# line-up
players_mins_DF.loc[2576335].loc[3162].players_id[-11:]

In [ ]:
players_mins_DF

In [ ]:
aux_filtered = filterDB(passes,partidos_a[2576336],players_mins_DF.loc[2576336].loc[3158].players_id[-11:])
count = len(aux_filtered)
acc = np.mean(aux_filtered.accuracy)
kp = sum(aux_filtered.keyPass)
print(count,acc,kp)

In [ ]:
matches_wUltPartidos.head(1)

In [ ]:
# creamos feature de pases y disparos segun el lineup-11

shots_11H, shots_acc_11H, goals_H = [], [], []
shots_11A, shots_acc_11A, goals_A = [], [], []

passes_11H, passes_acc_11H, keyPasses_H = [], [], []
passes_11A, passes_acc_11A, keyPasses_A = [], [], []

for m in matches_wUltPartidos.reset_index().itertuples():
    #shots
    db_filtered_h = filterDB(shots,partidos_h[m.wyId],players_mins_DF.loc[(m.wyId,m.teamId_home)].players_id[-11:])
    shots_11H.append(len(db_filtered_h))
    shots_acc_11H.append(np.mean(db_filtered_h.accuracy))
    goals_H.append(sum(db_filtered_h.goal))
    db_filtered_a = filterDB(shots,partidos_a[m.wyId],players_mins_DF.loc[m.wyId].loc[m.teamId_away].players_id[-11:])
    shots_11A.append(len(db_filtered_a))
    shots_acc_11A.append(np.mean(db_filtered_a.accuracy))
    goals_A.append(sum(db_filtered_a.goal))

    #passes
    db_filtered_h = filterDB(passes,partidos_h[m.wyId],players_mins_DF.loc[m.wyId].loc[m.teamId_home].players_id[-11:])
    passes_11H.append(len(db_filtered_h))
    passes_acc_11H.append(np.mean(db_filtered_h.accuracy))
    keyPasses_H.append(sum(db_filtered_h.keyPass))
    db_filtered_a = filterDB(passes,partidos_a[m.wyId],players_mins_DF.loc[m.wyId].loc[m.teamId_away].players_id[-11:])
    passes_11A.append(len(db_filtered_a))
    passes_acc_11A.append(np.mean(db_filtered_a.accuracy))
    keyPasses_A.append(sum(db_filtered_a.keyPass))



In [ ]:
assert len(shots_11H) == len(shots_11A) == len(shots_acc_11H)

In [ ]:
matches_wUltPartidos['shots_11H'] = shots_11H
matches_wUltPartidos['shots_11A'] = shots_11A
matches_wUltPartidos['shots_acc_11H'] = shots_acc_11H
matches_wUltPartidos['shots_acc_11A'] = shots_acc_11A
matches_wUltPartidos['goals_H'] = goals_H
matches_wUltPartidos['goals_A'] = goals_A
matches_wUltPartidos['passes_11H'] = passes_11H
matches_wUltPartidos['passes_11A'] = passes_11A
matches_wUltPartidos['passes_acc_11H'] = passes_acc_11H
matches_wUltPartidos['passes_acc_11A'] = passes_acc_11A
matches_wUltPartidos['keyPasses_H'] = keyPasses_H
matches_wUltPartidos['keyPasses_A'] = keyPasses_A

In [ ]:
matches_wUltPartidos.columns

In [ ]:
print(matches_wUltPartidos.loc[2576335].acc_avg_sh_A, matches_wUltPartidos.loc[2576335].shots_acc_11A)

In [ ]:
# matches_wUltPartidos.to_csv(path_rawdata+'matches_wUltPartidos.csv',sep=';')
matches_wUltPartidos = pd.read_csv(path_rawdata+'matches_wUltPartidos.csv',sep=';',index_col='wyId')
matches_wUltPartidos

In [ ]:
matches_wUltPartidos.mins4_H

In [ ]:
matches_wUltPartidos.columns

## INDICADOR ATAQUE-DEFENSA

avg. goles marcados / avg. goles encajados   [ ult. 4 partidos ]

In [ ]:
teams.loc[1673]

In [ ]:
partidos_h

In [ ]:
matches_stats.head(2)

In [ ]:
ataque_defensa = [[],[]]

for match in matches_wUltPartidos.itertuples(): 
    df_ult_partH = matches_stats.loc[partidos_h[match.matchId]][['teamId_home','teamId_away','score_away','score_home']]
    df_ult_partA = matches_stats.loc[partidos_a[match.matchId]][['teamId_home','teamId_away','score_away','score_home']]

    df_ult = [df_ult_partH, df_ult_partA]
    col    = ['teamId_home','teamId_away']
    score  = ['score_away','score_home']

    for i,df in enumerate(df_ult):
        # avg_encajados, avg_marcados = np.mean(df[score[i]]), np.mean(df[score[not(i)]])
        avg_encajados, avg_marcados = np.sum(df[score[i]]), np.mean(df[score[not(i)]])
        if(avg_encajados > 0):
            ataque_defensa[i].append(avg_marcados/avg_encajados)   
        else: ataque_defensa[i].append(avg_marcados)

# un equipo que solo ha recibido un gol y ha marcado X goles recibirá mejor puntuación
# que otro equipo que ha marcado los mismos goles pero no ha encajado ni un gol.
# si dividimos por la SUMA de los goles encajados desaparece este problema.
# sin embargo, un equipo que recibe un solo gol y no marca ninguno, 
# recibe menos puntuacion que otro que marca un gol y recibe muchos.

In [ ]:
assert len(ataque_defensa[0]) == len(matches_wUltPartidos)

In [ ]:
ataque_defensa_h = np.array(ataque_defensa[0])
ataque_defensa_a = np.array(ataque_defensa[1])

matches_wUltPartidos['ataque_defensa_h'] = (ataque_defensa_h-np.mean(ataque_defensa_h))/np.std(ataque_defensa_h)
matches_wUltPartidos['ataque_defensa_a'] = (ataque_defensa_a-np.mean(ataque_defensa_a))/np.std(ataque_defensa_a)

In [ ]:
max(matches_wUltPartidos.ataque_defensa_h)

In [ ]:
ataque_defensa_h.max()

In [ ]:
matches_wUltPartidos.to_csv(path_rawdata+'matches_wUltPartidos.csv',sep=';')

In [ ]:
avg_encajados_home = matches_wUltPartidos.groupby('teamId_home').agg(encajados=('score_away','mean'))
avg_marcados_home  = matches_wUltPartidos.groupby('teamId_home').agg(marcados=('score_home','mean'))
avg_encajados_away = matches_wUltPartidos.groupby('teamId_away').agg(encajados=('score_home','mean'))
avg_marcados_away  = matches_wUltPartidos.groupby('teamId_away').agg(marcados=('score_away','mean'))

avg_encajados = (pd.concat([avg_encajados_away,avg_encajados_home],axis=1)).mean(axis=1)  # (encajados como locales + commo encajados visit) / 2 -> pq son mismo # partidos
avg_marcados  = (pd.concat([avg_marcados_home,avg_marcados_away],axis=1)).mean(axis=1)

ataque_defensa = (avg_marcados/avg_encajados).sort_index()
ataque_defensa_norm = (ataque_defensa-ataque_defensa.mean())/ataque_defensa.std()

ataque_defensa_teams = pd.DataFrame(ataque_defensa_norm,columns=['indicador']).reset_index()
teamId = ataque_defensa_teams['index']
ataque_defensa_teams['teamId'] = teamId
ataque_defensa_teams.drop('index',axis=1,inplace=True)


In [ ]:
ataque_defensa_teams.set_index('teamId',inplace=True)

In [ ]:
ataque_defensa_teams

In [ ]:
ad_h, ad_a = [], []

for match in matches_wUltPartidos.itertuples():
    ind_h = ataque_defensa_teams.loc[match.teamId_home].indicador
    ind_a = ataque_defensa_teams.loc[match.teamId_away].indicador
    ad_h.append(ind_h), ad_a.append(ind_a)
    
matches_wUltPartidos['ataque_defensa_season_h'] = ad_h
matches_wUltPartidos['ataque_defensa_season_a'] = ad_a

In [ ]:
matches_wUltPartidos